# Project: Gene Sequence Results
#### Laurent Ehrlich: ehrlich@medicine.tamhsc.edu

This iPython Notebook serves to describe the computational steps we performed in analyzing the gene sequence data. Each section describes one python script, and the sections also itemize, in sequential order, the steps of the analysis. 

___

## Script: analyze_isoforms.py
The python script titled "analyze_isoforms.py" first opens the text file "file_manifest.txt". Below the first 6 rows of "file_manifest.txt" are printed, e.g. the rows which corresponds to Sample TCGA-ZH-A8Y4-01. The columns titled "Platform Type", "Center", "Platform", "Level" can be collated together with backslashes to form the path directory where the files in the column title "File Name" can be found: RNASeqV2/UNC__IlluminaHiSeq_RNASeqV2/Level_3/.

In [1]:
#import analyze_isoforms

In [27]:
file_manifest.iloc[:6]

,Platform Type,Center,Platform,Level,Sample,Barcode,File Name
0,RNASeqV2,UNC,IlluminaHiSeq_RNASeqV2,3,TCGA-ZH-A8Y4-01,TCGA-ZH-A8Y4-01A-11R-A41I-07,unc.edu.012cd145-9f6c-4b79-b929-3f42f61e3dce./...
1,RNASeqV2,UNC,IlluminaHiSeq_RNASeqV2,3,TCGA-ZH-A8Y4-01,TCGA-ZH-A8Y4-01A-11R-A41I-07,unc.edu.012cd145-9f6c-4b79-b929-3f42f61e3dce.2...
2,RNASeqV2,UNC,IlluminaHiSeq_RNASeqV2,3,TCGA-ZH-A8Y4-01,TCGA-ZH-A8Y4-01A-11R-A41I-07,unc.edu.012cd145-9f6c-4b79-b929-3f42f61e3dce.2...
3,RNASeqV2,UNC,IlluminaHiSeq_RNASeqV2,3,TCGA-ZH-A8Y4-01,TCGA-ZH-A8Y4-01A-11R-A41I-07,unc.edu.012cd145-9f6c-4b79-b929-3f42f61e3dce.2...
4,RNASeqV2,UNC,IlluminaHiSeq_RNASeqV2,3,TCGA-ZH-A8Y4-01,TCGA-ZH-A8Y4-01A-11R-A41I-07,unc.edu.012cd145-9f6c-4b79-b929-3f42f61e3dce.2...
5,RNASeqV2,UNC,IlluminaHiSeq_RNASeqV2,3,TCGA-ZH-A8Y4-01,TCGA-ZH-A8Y4-01A-11R-A41I-07,unc.edu.012cd145-9f6c-4b79-b929-3f42f61e3dce.2...


For a given sample, its "\*rsem\*genes.results\*" file matches its Gene IDs to its Transcript IDs, and its "\*rsem\*isoform.normalized_results\*" file gives the Normalized Count for its Isoform IDs. The "analyze_isoforms.py" script reads these "\*rsem\*" files into python as pandas DataFrames and merges them into a single table by matching Transcript IDs to Isoform IDs. The resulting table is named "isoforms" and is written to "geneSequenceResults.db", our SQLite3 database file.

In [30]:
print(genes_results.head())
print("\n")
print(isoforms_normalized_results.head())
print("\n")
merged_genes_isoforms.head()

       gene_id  raw_count  scaled_estimate          transcript_id
0  ?|100130426       0.00     0.000000e+00             uc011lsn.1
1  ?|100133144       6.42     2.105984e-07  uc010unu.1,uc010uoa.1
2  ?|100134869      19.58     4.641683e-07  uc002bgz.2,uc002bic.2
3      ?|10357     154.78     1.179019e-05             uc010zzl.1
4      ?|10431    2264.00     7.746698e-05  uc001jiu.2,uc010qhg.1


   isoform_id  normalized_count
0  uc011lsn.1            0.0000
1  uc010unu.1            3.4286
2  uc010uoa.1            0.0000
3  uc002bgz.2           10.4567
4  uc002bic.2            0.0000




,geneid,isoformid,normcount,barcode,tissuetype
0,?|100130426,uc011lsn.1,0.0000,TCGA-ZH-A8Y4-01A-11R-A41I-07,TN
1,?|100133144,uc010unu.1,3.4286,TCGA-ZH-A8Y4-01A-11R-A41I-07,TN
2,?|100133144,uc010uoa.1,0.0000,TCGA-ZH-A8Y4-01A-11R-A41I-07,TN
3,?|100134869,uc002bgz.2,10.4567,TCGA-ZH-A8Y4-01A-11R-A41I-07,TN
4,?|100134869,uc002bic.2,0.0000,TCGA-ZH-A8Y4-01A-11R-A41I-07,TN


## Script: analyze_genes.py
Similarly, the python script titled "analyze_genes.py" performs the analogous file manipulation and dataframe merges for the Gene data. The results are written to the same "geneSequenceResults.db" under a new table named "genes".

In [2]:
#import analyze_genes

___

## Script: match_isoform_counts.py
Picking up where the previous scripts left off, the "match_isoform_counts.py" script finds entries with the same Gene ID, Isoform ID, and Sample, and aligns their TN and NT norm counts onto the same row. We write the results to a table named "isoform_pairs" in database "geneSequenceResults.db".

In [18]:
#import match_isoform_counts

In [46]:
pandas.merge(NT_check, TN_check, how='outer')

,geneid,isoformid,normcount,barcode,tissuetype,sample
0,?|100130426,uc011lsn.1,0.0000,TCGA-W5-AA2R-11A-11R-A41I-07,NT,TCGA-W5-AA2R
1,?|100130426,uc011lsn.1,0.5111,TCGA-W5-AA2R-01A-11R-A41I-07,TN,TCGA-W5-AA2R


In [52]:
pandas.merge(NT_check, TN_check, on=['geneid','isoformid','sample'], how='inner', suffixes=('_NT', '_TN'))[['isoformid','sample','geneid','normcount_TN','normcount_NT']]

,isoformid,sample,geneid,normcount_TN,normcount_NT
0,uc011lsn.1,TCGA-W5-AA2R,?|100130426,0.5111,0


## Script: match_gene_counts.py
Similarly, the python script titled "match_gene_counts.py" performs the analogous TN/NT norm counts merges for the Gene data. The results are written to "geneSequenceResults.db" under the table name "gene_pairs".

In [16]:
#import match_gene_counts

___

## Script: paired_ttest_genes.py

The script "paired_ttest_genes.py" performs a paired t-test on each Gene ID's 9 samples where the paired observations are the NT and TN norm counts, returning the computed t-values and p-values and storing these results in the table "gene_paired_ttest" within "geneSequenceResults.db".

**WARNING:** The computational run time for this script is ~3 hours.

In [4]:
#import paired_ttest_genes

In [59]:
pandas.read_sql_query("select * from gene_paired_ttest limit 10", con)

,geneid,tvalue,pvalue
0,?|100130426,1.000000,0.346594
1,?|100133144,2.641350,0.029651
2,?|100134869,2.976060,0.017707
3,?|10357,0.680083,0.515646
4,?|10431,2.038709,0.075826
5,?|136542,NaN,NaN
6,?|155060,2.542636,0.034569
7,?|26823,3.495675,0.008130
8,?|280660,NaN,NaN
9,?|317712,NaN,NaN


## Script: paired_ttest_isoforms.py
Similarly, the python script titled "paired_ttest_isoforms.py" performs the analogous paired t-test on each Isoform ID's 9 samples of NT and TN norm counts. The results are written to "geneSequenceResults.db" under the table name "isoform_pairs".

**WARNING:** The computational run time for this script is ~9 hours.

In [17]:
#import paired_ttest_isoforms

___

## Script: correlations_genes.py

To run the script "correlations_genes.py" you must specify two command line arguments: the first is the gene id of interest, and the second is the alpha-level of correlation significance you wish to detect. In this example, we specify the 'MEN1' gene and a $\alpha$=0.1 significance-level.

The script "correlations_genes.py" uses the differences between NT and TN norm counts of a Gene ID's 9 samples to compute its sample correlation with that of the MEN1 gene. This script then determines which correlation values are statistically significant by calculating their p-values and using the Holm-Bonferroni adjustment with a specified $\alpha$-level. Results are written to the table "genes_significant_with_men1" in "geneSequenceResults.db".

**Holm-Bonferroni:** Because we are testing 20,531 hypotheses, we must consider that this increases the chances of randomly observing a high correlation and erroneously classifying it as 'significant'. To correct for this, we use the Holm-Bonferroni adjustment for Multiple Comparisons which improves the power of our hypotheses testing.

In [15]:
#import correlations_genes


Number of significantly correlated genes: 1

              p-values  p-adjusted 0.1-significant
geneid                                            
MRPL16|54948  0.000004    0.075452            True


## Script: correlations_isoforms.py
Similarly, the python script titled "correlations_isoforms.py" performs the analogous significance correlation analysis of each of MEN1's 9 isoforms against the 20,531 genes. The results are written to "geneSequenceResults.db" under the table name "genes_significant_with_men1_isoforms".

In [ ]:
#import correlations_isoforms

___